# Import Libraries

In [9]:
import torch, torchvision
from torch import nn
from torch import optim
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import matplotlib.pyplot as plt

import copy

from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import random 


BATCH_SIZE = 4

NameError: name '_C' is not defined

# Getting Data

In [ ]:

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_data = torchvision.datasets.MNIST('mnist_data', train = True, download = True, transform = transform)

test_data = torchvision.datasets.MNIST('mnist_data', train = False, download = True, transform = transform)

train_dl = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE)
test_dl = torch.utils.data.DataLoader(test_data, batch_size = BATCH_SIZE)



In [ ]:
plt.imshow(train_data[0][0][0], cmap = "gray")

# Create The Model

In [ ]:
def create_lenet():
    model = nn.Sequential(
        
    nn.Conv2d(1, 6, 5, padding = 2),
    nn.ReLU(),
    nn.AvgPool2d(2, stride = 2),
    
    nn.Conv2d(6, 16, 5, padding = 0),
    nn.ReLU(),
    nn.AvgPool2d(2, stride = 2), 
    
    nn.Flatten(),
    nn.Linear(400, 120),
    nn.ReLU(),
    nn.Linear(120, 84),
    nn.ReLU(),
    nn.Linear(84, 10)
)
    return model

# Validate The Model

In [ ]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        x = model(images)
        value, pred = torch.max(x, 1)
        total += x.size(0)
        correct += torch.sum(pred == labels)
    
    return correct*100./total

# Training Function

In [ ]:
def train(lr, epochs = 3, device = "cpu"):
    accuracies = []
    cnn = create_lenet().to(device)
    cec = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr = lr)
    max_accuracy = 0
    
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_dl):
            optimizer.zero_grad()
            pred = cnn(images)
            loss = cec(pred, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(cnn, test_dl))
        accuracies.append(accuracy)
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(cnn)
            max_accuracy = accuracy 
            print("Saving Best Model With Accuracy: ", accuracy)
        print("Epoch: ", epoch+1, "Accuracy: ", accuracy, "%")
    #plt.plot(accuracies)
    return best_model, accuracies, lr

# Training The Model: Good Learning Rates

In [ ]:
lenet = train(0.001, 10)

In [ ]:
print("Highest Accuracy for Lenet: ", max(lenet1[1]), " with learning rate: ", lenet1[2])


In [ ]:
x = [1,2,3,4,5,6,7,8,9,10]
plt.plot(x, lenet[1])

In [ ]:
def predict_dl(model, data):
    y_pred = []
    y_true = []
    for i, (images, labels) in enumerate(data):
        x = model(images)
        value, pred = torch.max(x, 1)
        pred = pred.data.cpu()
        y_pred.extend(list(pred.numpy()))
        y_true.extend(list(labels.numpy()))
    return np.array(y_pred), np.array(y_true)

In [ ]:
y_pred, y_true = predict_dl(lenet2[0], test_dl)

In [ ]:
pd.DataFrame(confusion_matrix(y_true, y_pred, labels = np.arange(0, 10)))